# Before Improvement

In [10]:
A = ['Hewlett-Packard', 'Google', 'Microsoft']
B = ['Microsoft Corporation', 'Hewlett Packard', 'gogle', 'miccro softt', 'HP',
     'Microsoft Co.', 'helwet pacard inc', 'microsoft company', 'Gooooogle', 'GOOGLE LLC', 'google inc..',
     'miccrosoftt', 'Hewlettt.Paccard', 'Hewlett.Packard.Company', 'micr soft corporation']

In [11]:
def get_levenshtein_distance(s1, s2, substitution_cost=1):
    '''
    computes the Levenshtein distance between two strings: the minimum total cost of insertions(cost = 1), deletions (cost = 1), and substitutions 
    (substitution_cost is set in the function) needed to transform one string into the other
    '''
    l1 = len(s1)
    l2 = len(s2)
    
    if l1 == 0:
        return l2
    if l2 == 0:
        return l1
    
    previous = [i for i in range(l2 + 1)]
    current = [0] * (l2 + 1)

    for i in range(l1):
        current[0] = i+1
        for j in range(l2):
            if(s1[i] == s2[j]):
                cost=0
            else:
                cost=substitution_cost
            
            current[j+1] = min(current[j] + 1, previous[j+1] + 1, previous[j] + cost)
        previous = current.copy()
    
    return current[l2]


def get_indel_distance(s1, s2):
    '''
    computes the total number of deletions and insertions needed to transform one string into the other
    '''
    return get_levenshtein_distance(s1, s2, 2)


def get_distance_normalized(s1, s2):
    '''
    computes the normalized similarity between two strings based on the number of deletions and insertions needed to transform one string into the other
    if the returnes value is close to 1, the strings are dissimilar
    if the returned value is close to 0, the strings are similar
    '''
    return 1-get_indel_distance(s1, s2)/(len(s1)+len(s2))
    

In [12]:
def preprocess_string(s):
    '''
    lowercase all symbols
    delete stopwords
    leave only words (a-z, A-Z, 0-9 symbols)
    '''
    s = s.lower()
    stopwords = ['company','corporation',  'llc', 'co', 'inc']
    for stopword in stopwords:
        s = s.replace(stopword, '')
    s = ''.join([c for c in s if c.isalnum()])
    return s

In [13]:
for a in A:
    print(a)
    a = preprocess_string(a)
    distances = [(b, get_distance_normalized(a, preprocess_string(b))) for b in B]
    distances.sort(key=lambda x: x[1], reverse=True)
    for i in range(3):
        print('\t', distances[i][0], distances[i][1])
    print()

Hewlett-Packard
	 Hewlett Packard 1.0
	 Hewlett.Packard.Company 1.0
	 Hewlettt.Paccard 0.896551724137931

Google
	 GOOGLE LLC 1.0
	 google inc.. 1.0
	 gogle 0.9090909090909091

Microsoft
	 Microsoft Corporation 1.0
	 Microsoft Co. 1.0
	 microsoft company 1.0



# edge case

In [14]:
get_distance_normalized("National Aeronautics and Space Administration", preprocess_string("NASA"))

0.16326530612244894

In [15]:
get_distance_normalized("NatWest", preprocess_string("NASA"))

0.36363636363636365

# Improvement

In [16]:
def get_abbreviation(s):
    '''
    returns the abbreviation of a string (example: National Aeronautics and Space Administration -> nasa) 
    if the string has only 1 word then the abbreviation is the word itself
    '''

    s = s.lower()

    stopwords = ['and', 'the', 'of', 'corporation', 'company', 'llc', 'inc', 'co']

    for stopword in stopwords:
        s = s.replace(stopword, '')

    # edge case: if the string is already an abbreviation or has no abbreviation
    if len(s.split()) == 1:
        return s
    
    s_first_letters = ''.join([word[0] for word in s.split()])

    return s_first_letters

In [17]:
def get_simularity_coef_improved(s1, s2):
    '''
    computes the normalized similarity between two company names based on the number of deletions 
    and insertions needed to transform one string into the other (taking into account the abbreviation of the company names)
    if the returnes value is close to 1, the strings are dissimilar
    if the returned value is close to 0, the strings are similar
    '''
    #check all 4 options and return the maximum value
    return max(get_distance_normalized(preprocess_string(s1), preprocess_string(s2)),
                get_distance_normalized(get_abbreviation(s1), get_abbreviation(s2)),
                get_distance_normalized(preprocess_string(s1), get_abbreviation(s2)),
                get_distance_normalized(get_abbreviation(s1), preprocess_string(s2)))

In [18]:
A = ['Hewlett-Packard', 'Google', 'Microsoft', 'NASA', 'NatWest', 'IBM']
B = ['Microsoft Corporation', 'Hewlett Packard', 'gogle', 'miccro softt', 'HP',
     'Microsoft Co.', 'helwet pacard inc', 'microsoft company', 'Gooooogle', 'GOOGLE LLC', 'google inc..',
     'miccrosoftt', 'Hewlettt.Paccard', 'Hewlett.Packard.Company', 'micr soft corporation', 'International Business Machines Corporation', 'National Aeronautics and Space Administration', 'NatWest Bank',
     'IBM Corporation', 'NatWest Group', 'NatWest Markets', 'NatWest Markets Plc', 'NatWest Markets Securities Inc', 'National Aeronautics and Space Administration (NASA)']

for a in A:
    print(a)
    distances = [(b, get_simularity_coef_improved(a, b)) for b in B]
    distances.sort(key=lambda x: x[1], reverse=True)
    for i in range(10):
        print('\t', distances[i][0], distances[i][1])
    print()

Hewlett-Packard
	 Hewlett Packard 1.0
	 Hewlett.Packard.Company 1.0
	 Hewlettt.Paccard 0.896551724137931
	 helwet pacard inc 0.8461538461538461
	 NatWest Bank 0.4
	 NatWest Markets Plc 0.3870967741935484
	 NatWest Markets 0.3571428571428571
	 NatWest Markets Securities Inc 0.3157894736842105
	 NatWest Group 0.3076923076923077
	 National Aeronautics and Space Administration 0.2909090909090909

Google
	 GOOGLE LLC 1.0
	 google inc.. 1.0
	 gogle 0.9090909090909091
	 Gooooogle 0.8
	 Microsoft Corporation 0.2666666666666667
	 Microsoft Co. 0.2666666666666667
	 microsoft company 0.2666666666666667
	 NatWest Group 0.25
	 miccro softt 0.23529411764705888
	 miccrosoftt 0.23529411764705888

Microsoft
	 Microsoft Corporation 1.0
	 Microsoft Co. 1.0
	 microsoft company 1.0
	 micr soft corporation 0.9411764705882353
	 miccro softt 0.9
	 miccrosoftt 0.9
	 NatWest Markets Securities Inc 0.4
	 NatWest Markets 0.2857142857142857
	 GOOGLE LLC 0.2666666666666667
	 google inc.. 0.2666666666666667

NASA
	 